# TensorFlowでのモデルサイズ推定

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

## モデルサイズ計算関数

下記のページのプログラム
https://stackoverflow.com/questions/43137288/how-to-determine-needed-memory-of-keras-model
では、混合精度演算には対応していないため、モデル中の各レイヤのdtype_policyを元にデータサイズを推定するように修正を試みた。

In [2]:
def get_model_memory_usage_mixed_mode(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    def get_bytes(dtype_policy_name):
        if dtype_policy_name == 'float16':
            return 2
        elif dtype_policy_name == 'float32':
            return 4
        elif dtype_policy_name == 'float64':
            return 8
        if dtype_policy_name == 'mixed_float16':
            return 2
        if dtype_policy_name == 'mixed_bfloat16':
            return 2
        else:
            raise "unknow dtype_plocy;{}".format(dtype_policy_name)
    
    internal_model_mem_count = 0
    trainable_count = 0
    non_trainable_count = 0
    total_memory = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage_mixed_mode(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
 
        trainable_count = np.sum([K.count_params(p) for p in l.trainable_weights])
        non_trainable_count = np.sum([K.count_params(p) for p in l.non_trainable_weights])

        total_memory += get_bytes(l.dtype_policy.name) * batch_size * single_layer_mem * 2 # for Back Propagation
        total_memory += get_bytes(l.dtype_policy.name) * (trainable_count + non_trainable_count) * 2 # for Back Propagation
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

## PointNetモデルの場合

下記のページより  
https://keras.io/examples/vision/pointnet/

In [3]:
NUM_POINTS = 650000
NUM_CLASSES = 10

In [4]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


In [5]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [6]:
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [7]:
inputs = keras.Input(shape=(NUM_POINTS, 6))

x = tnet(inputs, 6)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

x = layers.Dense(NUM_CLASSES, activation="softmax")(x)
outputs = layers.Activation("softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()


Model: "pointnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 650000, 6)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 650000, 32)   224         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 650000, 32)  128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 650000, 32)   0           ['batch_normalization[0][0

In [9]:
for batch_size in [4, 8, 16, 32]:
  mem_size = get_model_memory_usage_mixed_mode(batch_size, model)
  print("Batch Size:{} , Memory Usage: {} GB".format(batch_size, mem_size))

Batch Size:4 , Memory Usage: 110.578 GB
Batch Size:8 , Memory Usage: 221.151 GB
Batch Size:16 , Memory Usage: 442.297 GB
Batch Size:32 , Memory Usage: 884.588 GB


## 3D CNNの場合
下記のページより  
https://keras.io/examples/vision/3D_image_classification/

In [10]:
def get_model(width=128, height=128, depth=64, channel=1):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, channel))

    x = layers.Conv3D(filters=64, kernel_size=3)(inputs)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(units=10)(x)
    outputs = layers.Activation(tf.keras.activations.softmax)(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=150, height=300, depth=150, channel=3)
model.summary()

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 300, 150, 3  0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 148, 298, 148, 64  5248      
                             )                                   
                                                                 
 activation_18 (Activation)  (None, 148, 298, 148, 64  0         
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 74, 149, 74, 64)  0         
 )                                                               
                                                                 
 batch_normalization_17 (Bat  (None, 74, 149, 74, 64)  256   

In [11]:
for batch_size in [4, 8, 16, 32]:
  mem_size = get_model_memory_usage_mixed_mode(batch_size, model)
  print("Batch Size:{} , Memory Usage: {}GB".format(batch_size, mem_size))

Batch Size:4 , Memory Usage: 32.723GB
Batch Size:8 , Memory Usage: 65.435GB
Batch Size:16 , Memory Usage: 130.86GB
Batch Size:32 , Memory Usage: 261.711GB


### 3D-CNN 512x512x512 の場合

In [12]:
# Build model.
model = get_model(width=512, height=512, depth=512, channel=3)
model.summary()

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 512, 512, 512, 3  0         
                             )]                                  
                                                                 
 conv3d_4 (Conv3D)           (None, 510, 510, 510, 64  5248      
                             )                                   
                                                                 
 activation_23 (Activation)  (None, 510, 510, 510, 64  0         
                             )                                   
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 255, 255, 255, 64  0        
 3D)                         )                                   
                                                                 
 batch_normalization_21 (Bat  (None, 255, 255, 255, 64  256  

In [13]:
for batch_size in [4, 8, 16, 32]:
  mem_size = get_model_memory_usage_mixed_mode(batch_size, model)
  print("Batch Size:{} , Memory Usage: {}GB".format(batch_size, mem_size))

Batch Size:4 , Memory Usage: 670.766GB
Batch Size:8 , Memory Usage: 1341.522GB
Batch Size:16 , Memory Usage: 2683.034GB
Batch Size:32 , Memory Usage: 5366.058GB
